<hr style="height:3px;border:none;color:#333;background-color:#333;" />
<img style=" float:right; display:inline" src="http://opencloud.utsa.edu/wp-content/themes/utsa-oci/images/logo.png"/>

### **University of Texas at San Antonio** 
<br/>
<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2.5em;"> **Open Cloud Institute** </span>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### OpenStack Cloud Networking

<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Paul Rad, Ph.D.** </span>  


<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Gonzalo De La Torre, Ph.D. Student** </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> *Open Cloud Institute, University of Texas at San Antonio, San Antonio, Texas, USA* </span>  
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> {gonzalo.delatorreparra, paul.rad}@utsa.edu </span>  

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

# Cloud Networking
In today's data centers, networks are composed of more devices than ever before.
Servers, switches, routers, storage systems, and security appliances now exist
as virtual machines and virtual network appliances. These devices place a large
strain on traditional network management systems, as they are unable to provide
a scalable, automated approach to managing next-generation networks. Users now
expect more control and flexibility of the infrastructure with quicker provisioning,
all of which Cloud Networking promises to delive
## Functions Of Cloud Networking
Cloud Networking includes many technologies one would find in a data center,
including (**switching, routing, load balancing, firewalling, and virtual private networks**)aaS.
These features can be configured to leverage open source or commercial software,
and provide a cloud operator with all the tools necessary to build a functional and
self-contained cloud.Cloud Networking also provides a framework for
third-party vendors to build on and enhance the capabilities of the cloud


## Neutron At A Glance

* Give cloud tenants an API to build rich networking topologies, and configure advanced network policies in the cloud.
 * Example: create multi-tier web application topology
* Enable innovation plugins (open and closed source) that introduce advanced network capabilities
 * Example: use L2-in-L3 tunneling to avoid VLAN limits, provide end-to-end QoS guarantees, use monitoring protocols like NetFlow.
* Let anyone build advanced network services (open and closed source) that plug into Openstack tenant networks.
 * Examples: LB-aaS, VPN-aaS, firewall-aaS, IDS-aaS (not implemented), data-center-interconnect-aaS.
* Horizon GUI support for:
 * Neutron L2 and L3 network and subnet creation/deletion
 * Booting VMs on specific Neutron networks.
* API Extensibility Framework, including extensions for:
 * "provider network", which maps Neutron L2 networks to a specific VLAN in the physical data center


## Network Topologies Using Neutron
 Network Topologies using Neutron is an architecture which describes the network  which includes the private and public networks hsving their interefaces which are connected each other with L3 and L2 devices.



## Cleanig The Default Network Topology
Chameleon Cloud provide user the by defalut public and private network which we can delete by following command and further we will create our own Environment.

In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin
ROUTER_ID=$(neutron router-list | grep router1 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| private-subnet' | awk '{print $2}')
PUBLIC_SUBNET_ID=$(neutron subnet-list | grep '| public-subnet' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  neutron port-update --device-owner None $i;
  neutron port-delete $i;
done;

neutron router-delete router1
neutron net-delete public
neutron net-delete private


## 1st Basic Network Topology

The best way of learining and keeping that for a long is by starting with the basics, so using the following commands user can create the basic topology in which we will connect a inatance with the private network.



![Image](https://github.com/ml6973/Cloud-Tutorial/blob/master/OpenStack/figures/case1.png?raw=True)



In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

neutron net-create my-private-network-1
neutron net-create my-private-network-2
neutron subnet-create my-private-network-1 --name my-private-subnet-1 --gateway 192.177.0.1 192.177.0.0/24
neutron subnet-create my-private-network-2 --name my-private-subnet-2 --gateway 192.178.0.1 192.178.0.0/24
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-1 my-instance-1
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-2 my-instance-2

PRIVATE_NET1_ID=$(neutron net-list | grep '| my-private-network-1' | awk '{print $2}')

PRIVATE_NET1_NS='qdhcp-'$PRIVATE_NET1_ID

NOVA_1=$(nova list | grep '| my-private-network-1' | awk '{print $12}')

echo $NOVA_1

sudo ip netns exec $PRIVATE_NET1_NS sshpass -p "cubswin:)" ssh  cirros@$NOVA_1


In the given topology the instace is connected to the its individual private network but there is no way sos that the user can ping the one instance from the otehr instance.

## Cleaning-up The Basic Network Topology

Using the below given code user can delete the basic network topology.

In [ ]:
%%bash
. /home/stack/devstack/openrc admin admin

nova delete my-instance-1
nova delete my-instance-2

#Private 1,2
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet1' | awk '{print $2}')
for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  if [ "$i" != "|" ]
  then
    neutron port-update --device-owner None $i;
    neutron port-delete $i;
  fi
done;

neutron net-delete my-private-network-1
neutron net-delete my-private-network-2


#nova aggregate-remove-host my-aggregate-0 computenode0
#nova aggregate-delete my-aggregate-0

## One Step Ahead

In the above topology user can reach his his private netowork from his instance but it was not possible to reach to the another instance which is connected to the another private network. So to reslove that problem user can introduce a L3 device which is capable of transferring traffic.



![Image](https://github.com/ml6973/Cloud-Tutorial/blob/master/OpenStack/figures/case2.png?raw=True)



## 2nd Basic Network Topology

In [4]:
%%bash
. /home/stack/devstack/openrc admin admin
neutron net-create my-private-network-1
neutron net-create my-private-network-2
neutron subnet-create my-private-network-1 --name my-private-subnet-1 --gateway 192.180.0.1 192.180.0.0/24
neutron subnet-create my-private-network-2 --name my-private-subnet-2 --gateway 192.181.0.1 192.181.0.0/24

neutron router-create my-router-1

neutron router-interface-add my-router-1 my-private-subnet-1
neutron router-interface-add my-router-1 my-private-subnet-2

nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-1 my-instance-1
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-2 my-instance-2


Created a new network:
+---------------------------+--------------------------------------+
| Field                     | Value                                |
+---------------------------+--------------------------------------+
| admin_state_up            | True                                 |
| availability_zone_hints   |                                      |
| availability_zones        |                                      |
| created_at                | 2016-11-08T16:22:40                  |
| description               |                                      |
| id                        | f51dca72-57b9-4046-bf5d-e6aeb8f5eb62 |
| ipv4_address_scope        |                                      |
| ipv6_address_scope        |                                      |
| mtu                       | 1450                                 |
| name                      | my-private-network-1                 |
| port_security_enabled     | True                                 |
| provider:

## Cleaning-up 2nd Network Topology

In [5]:
%%bash
. /home/stack/devstack/openrc admin admin

nova delete my-instance-1
nova delete my-instance-2

#Router 1, Public, Private 1,2
ROUTER_ID=$(neutron router-list | grep my-router-1 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet1' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  if [ "$i" != "|" ]
  then
    neutron port-update --device-owner None $i;
    neutron port-delete $i;
  fi
done;

neutron router-delete my-router-1
neutron net-delete my-private-network-1
neutron net-delete my-private-network-2


#nova aggregate-remove-host my-aggregate-0 computenode0
#nova aggregate-delete my-aggregate-0






Request to delete server my-instance-1 has been accepted.
Request to delete server my-instance-2 has been accepted.
Removed gateway from router ac06eeb7-855c-4c66-a83d-e3dc9105ba94
Updated port: 125dbeeb-0fec-46f3-8199-0f1879a98fc1
Deleted port: 125dbeeb-0fec-46f3-8199-0f1879a98fc1
Updated port: 6fd14383-e372-43d1-a66e-fd6ed1929d98
Deleted port: 6fd14383-e372-43d1-a66e-fd6ed1929d98
Updated port: d076fc67-816c-4367-a6e0-15bfcd012198
Deleted port: d076fc67-816c-4367-a6e0-15bfcd012198
Updated port: daad6cf7-c73d-43e6-a250-d94d230fd995
Deleted port: daad6cf7-c73d-43e6-a250-d94d230fd995
Deleted router: my-router-1
Deleted network: my-private-network-1
Deleted network: my-private-network-2


## Creating our 1st Complex Network Topology
In our first complex topology we have one public network which is connected to the two router. Each routers are further connected to the two different private networks and two of them are connected to the instances.

![Image](https://github.com/cloudandbigdatalab/eLab-GUI-web-portal/blob/development/WebContent/resources/marketPlaceData/CloudNetworking/images/topology.png?raw=True)



## Creating The Topology Using Neutron CLI
Neutron CLI enables the user to create, delete and modify the netowks which is composed of public/private network and L2 and L3 network components.

In [1]:
%%bash
. /home/stack/devstack/openrc admin admin

neutron net-create --shared --provider:network_type flat --provider:physical_network flat --router:external my-public-network

neutron subnet-create --name my-public-subnet-1 --gateway 10.10.10.1 --allocation-pool start=10.10.10.2,end=10.10.10.255 --dns-nameserver 8.8.8.8 my-public-network 10.10.0.0/16
neutron subnet-create --name my-public-subnet-2 --gateway 10.20.10.1 --allocation-pool start=10.20.10.2,end=10.20.10.255 --dns-nameserver 8.8.8.8 my-public-network 10.20.0.0/16

neutron net-create my-private-network-1
neutron net-create my-private-network-2
neutron net-create my-private-network-3
neutron net-create my-private-network-4

neutron subnet-create my-private-network-1 --name my-private-subnet-1 --gateway 192.168.0.1 192.168.0.0/24
neutron subnet-create my-private-network-2 --name my-private-subnet-2 --gateway 192.169.0.1 192.169.0.0/24
neutron subnet-create my-private-network-3 --name my-private-subnet-3 --gateway 192.170.0.1 192.170.0.0/24
neutron subnet-create my-private-network-4 --name my-private-subnet-4 --gateway 192.171.0.1 192.171.0.0/24

neutron router-create my-router-1
neutron router-create my-router-2

neutron router-interface-add my-router-1 my-public-subnet-1
neutron router-interface-add my-router-2 my-public-subnet-2

neutron router-interface-add my-router-1 my-private-subnet-1
neutron router-interface-add my-router-1 my-private-subnet-2
neutron router-interface-add my-router-2 my-private-subnet-3
neutron router-interface-add my-router-2 my-private-subnet-4

nova flavor-list
glance image-list

nova aggregate-create my-aggregate-0 compute-0
nova aggregate-add-host my-aggregate-0 computenode0

nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone nova --nic net-name=my-private-network-1 my-instance-01
nova boot --flavor 1 --image cirros-0.3.4-x86_64-uec --security-groups default --availability-zone compute-0 --nic net-name=my-private-network-4 my-instance-02

nova list
nova availability-zone-list

Created a new network:
+---------------------------+--------------------------------------+
| Field                     | Value                                |
+---------------------------+--------------------------------------+
| admin_state_up            | True                                 |
| availability_zone_hints   |                                      |
| availability_zones        |                                      |
| created_at                | 2016-11-07T03:49:57                  |
| description               |                                      |
| id                        | f6f2c2f0-1fd4-4049-8aef-ecb1a1e5d558 |
| ipv4_address_scope        |                                      |
| ipv6_address_scope        |                                      |
| is_default                | False                                |
| mtu                       | 1500                                 |
| name                      | my-public-network                    |
| port_secu

## Output of The Code
Using the above code user have created the network topology.

## Cleaning-up Your Network Topology Using Neutron CLI
As we used the Neutron CLI to create the Network toplogy, we can use it do delete the created topology with the help of commands.


In [1]:
%%bash
. /home/stack/devstack/openrc admin admin

nova delete my-instance-01
nova delete my-instance-02

#Router 1, Public, Private 1,2
ROUTER_ID=$(neutron router-list | grep my-router-1 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet1' | awk '{print $2}')
PUBLIC_SUBNET_ID=$(neutron subnet-list | grep '| my-public-subnet' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  if [ "$i" != "|" ]
  then
    neutron port-update --device-owner None $i;
    neutron port-delete $i;
  fi
done;

neutron router-delete my-router-1
neutron net-delete my-public-network
neutron net-delete my-private-network-1
neutron net-delete my-private-network-2

#Router 2, Private 3,4
ROUTER_ID=$(neutron router-list | grep my-router-2 | grep -v '+' | grep -v 'external_gateway' | awk '{ print $2 }')
PRIVATE_SUBNET_ID=$(neutron subnet-list | grep '| my-private-subnet3' | awk '{print $2}')
neutron router-gateway-clear $ROUTER_ID

for i in `neutron port-list | grep -v '+' | grep -v 'fixed_ips' | awk '{print $2}'`;
do
  neutron port-update --device-owner None $i;
  neutron port-delete $i;
done;

neutron router-delete my-router-2
neutron net-delete my-private-network-3
neutron net-delete my-private-network-4

nova aggregate-remove-host my-aggregate-0 computenode0
nova aggregate-delete my-aggregate-0

Request to delete server my-instance-01 has been accepted.
Request to delete server my-instance-02 has been accepted.
Removed gateway from router 50690885-91b5-45a7-8e1e-91efe986790d
Updated port: 3b7391ea-7eed-4970-8531-fb152512dc90
Deleted port: 3b7391ea-7eed-4970-8531-fb152512dc90
Updated port: 48635891-a791-4726-98ec-d990f31b47a8
Deleted port: 48635891-a791-4726-98ec-d990f31b47a8
Updated port: 53133dfb-9701-4bf1-886b-4ce50527aa86
Deleted port: 53133dfb-9701-4bf1-886b-4ce50527aa86
Updated port: 8261a662-aca1-4210-859d-eb79ede305fd
Deleted port: 8261a662-aca1-4210-859d-eb79ede305fd
Updated port: 8681b14a-3f70-4498-81c8-56c9ca05c001
Deleted port: 8681b14a-3f70-4498-81c8-56c9ca05c001
Updated port: a62ef1ca-13e2-4658-b66a-e658fb928af8
Deleted port: a62ef1ca-13e2-4658-b66a-e658fb928af8
Updated port: bf569dea-b4f0-4a3d-afa0-64625d44cc3e
Deleted port: bf569dea-b4f0-4a3d-afa0-64625d44cc3e
Updated port: c20dd9d9-ce74-404e-8d92-32e315a62d97
Deleted port: c20dd9d9-ce74-404e-8d92-32e315a62d97
U

## Output 
Using the above code user have deleted the network topology.